# CS535 Pytorch Tutorial - Define a Convolutional Neural Network

### Construct Your Network
In pytorch any neual network is a subclass that derived from its superclass ```torch.nn.Module```.
Let's build a toy CNN network to see how this works.

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
        #                padding=0, dilation=1, groups=1, bias=True)
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        # torch.nn.Linear(in_features, out_features, bias=True)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #32x32 -> 28x28 -> 14x14 -> 10x10 -> 5x5
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # Now we have defined the above components of our network, the next
        # step is to connect them into the order we wanted
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2))
        # Or, you can put this into one line:
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Tensor.view() == numpy.reshape()
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


You just have to define the ```forward``` function, and the ```backward``` function (where gradients are computed) is automatically defined for you using autograd. You can use any of the Tensor operations in the forward function.

The learnable parameters of a model are returned by ```net.parameters()```

In [8]:
params = list(net.parameters())
print(len(params))
print([x.size() for x in params])  # weight and bias for each layers

10
[torch.Size([6, 1, 5, 5]), torch.Size([6]), torch.Size([16, 6, 5, 5]), torch.Size([16]), torch.Size([120, 400]), torch.Size([120]), torch.Size([84, 120]), torch.Size([84]), torch.Size([10, 84]), torch.Size([10])]


The input to the forward is an autograd.Variable, and so is the output. Note: Expected input size to this net(LeNet) is ```32x32```. To use this net on MNIST dataset,please resize the images from the dataset to ```32x32```.
In Pytorch your input must be feeded in mini-batch, even if you want to feed your data one by one, you have to use  ```input.unsqueeze(0)``` to expand your input dimensionality.

For 2D images, your input dimension should be: ```[batch_size, channel, height, width]```

In [28]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0249 -0.1065 -0.0304  0.1617 -0.0185 -0.0671  0.1201 -0.0193 -0.0295  0.0818
[torch.FloatTensor of size 1x10]



## Define Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: ```nn.MSELoss``` which computes the mean-squared error between the input and the target.

In [57]:
output = net(input)
target = Variable(torch.Tensor([3]).type(torch.LongTensor))
# a dummy target, for example
print(target)
# Log-Softmax-Crossentropy loss 
criterion = nn.CrossEntropyLoss()

loss = criterion(output, target)
print(output, target, loss)

Variable containing:
 3
[torch.LongTensor of size 1]

(Variable containing:
-0.0249 -0.1065 -0.0304  0.1617 -0.0185 -0.0671  0.1201 -0.0193 -0.0295  0.0818
[torch.FloatTensor of size 1x10]
, Variable containing:
 3
[torch.LongTensor of size 1]
, Variable containing:
 2.1510
[torch.FloatTensor of size 1]
)


Now, if you follow loss in the backward direction, using it’s .grad_fn attribute, you will see a graph of computations that looks like this:
```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> CrossEntropyLoss
      -> loss
```

So, when we call loss.backward(), the whole graph is differentiated w.r.t. the loss, and all Variables in the graph will have their .grad Variable accumulated with the gradient.

For illustration, let us follow a few steps backward

In [65]:
print(loss.grad_fn)  # CrossEntropyLoss
print(loss.grad_fn.next_functions[0][0])  # Log Softmax loss
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # Linear

### Back Propagation
To backpropagate the error all we have to do is to ```loss.backward()```. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients

Now we shall call ```loss.backward()```, and have a look at conv1’s bias gradients before and after the backward.

In [66]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
Variable containing:
1.00000e-03 *
 -4.2934
 -2.8786
 -3.2121
  3.1474
 -6.7055
 -3.3181
[torch.FloatTensor of size 6]



### Update the Weights
The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

```weight = weight - learning_rate * gradient```


We can implement this using simple python code:

In [67]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [68]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update